# Проект и задача

**Проект:**  
Прогнозирование почасового спроса на велопрокат. 

**Задача:**  
Разработать более точную модель прогноза спроса на велосипеды, учитывающую нелинейное влияние погодных и временных факторов.  
Сравнить результаты с базовой линейной регрессией и сохранить финальный пайплайн лучшей модели.


# 0. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

import optuna
import joblib

<b>Выводы: Импортированы все необходимые библиотеки для работы с данными, визуализации, моделирования (kNN, Decision Tree), предобработки, оценки метрик и оптимизации гиперпараметров (Optuna).</b>

# 1. Загрузка данных и baseline модели

In [ ]:
try:
    df_train = pd.read_csv(r'C:\Users\talantr\Desktop\kNN\datasets\ds_s14_train_data.csv')
    df_test = pd.read_csv(r'C:\Users\talantr\Desktop\kNN\datasets\ds_s14_test_data.csv')
    baseline_model = joblib.load(r'C:\Users\talantr\Desktop\kNN\datasets\baseline_linear_regression_pipeline.pkl')
    print("Данные и модель загружены из локального пути.")
except:
    df_train = pd.read_csv('/datasets/ds_s14_train_data.csv')
    df_test = pd.read_csv('/datasets/ds_s14_test_data.csv')
    baseline_model = joblib.load('/datasets/baseline_linear_regression_pipeline.pkl')
    print("Данные и модель загружены из универсального пути.")

<b>Выводы: Данные и baseline модель успешно загружены. Используются как локальные пути, так и универсальные для совместимости.</b>

# 2. Обзор данных

In [ ]:
df_train.info()
df_train.describe()
df_train.isna().sum()

In [ ]:
TARGET = 'Rented Bike Count'

X_train = df_train.drop(columns=[TARGET])
y_train = df_train[TARGET]
X_test = df_test.drop(columns=[TARGET])
y_test = df_test[TARGET]

<b>Выводы: - Датасет содержит 7008 записей и 16 признаков.
- Есть пропуски в некоторых числовых признаках (Humidity, Wind speed, Visibility, Solar Radiation, Rainfall, Snowfall).
- Целевая переменная: `Rented Bike Count`.
- Данные разделены на признаки и целевую переменную для train/test.</b>

# 3. Метрики для оценки моделей

In [ ]:
def regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        'RMSE': np.sqrt(mse),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred)
    }

baseline_pred_train = baseline_model.predict(X_train)
baseline_pred_test = baseline_model.predict(X_test)

baseline_train_metrics = regression_metrics(y_train, baseline_pred_train)
baseline_test_metrics = regression_metrics(y_test, baseline_pred_test)

In [ ]:
print("Baseline train metrics:", baseline_train_metrics)
print("Baseline test metrics:", baseline_test_metrics)

<b>Выводы: Определена функция для расчета метрик RMSE, MAE, R2.</b>

# 4. Предобработка данных

In [ ]:
numeric_features = X_train.select_dtypes(include='number').columns.tolist()
categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

<b>Выводы: - Числовые признаки обрабатываются через `SimpleImputer` (медиана) + `StandardScaler`.
- Категориальные признаки: `SimpleImputer` (most_frequent) + `OneHotEncoder`.
- Создан `ColumnTransformer` для автоматической предобработки.</b>

# 5. Определение пайплайнов моделей

In [ ]:
kNN_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor())
])

tree_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)

<b>Выводы: 
- Созданы пайплайны для kNN и Decision Tree.
- Определена кросс-валидация KFold (5 фолдов) для подбора гиперпараметров.</b>

# 6. Подбор гиперпараметров с Optuna

In [ ]:
# --- kNN ---
def knn_objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 2)

    model = Pipeline([
        ('preprocess', preprocessor),
        ('model', KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p))
    ])

    score = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv).mean()
    return -score

study_knn = optuna.create_study(direction='minimize')
study_knn.optimize(knn_objective, n_trials=30)
print("Лучшие параметры kNN:", study_knn.best_params)

In [ ]:
# --- Decision Tree ---
def tree_objective(trial):
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = Pipeline([
        ('preprocess', preprocessor),
        ('model', DecisionTreeRegressor(max_depth=max_depth,
                                        min_samples_split=min_samples_split,
                                        min_samples_leaf=min_samples_leaf,
                                        random_state=42))
    ])

    score = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv).mean()
    return -score

study_tree = optuna.create_study(direction='minimize')
study_tree.optimize(tree_objective, n_trials=30)
print("Лучшие параметры дерева решений:", study_tree.best_params)

<b>Выводы:
- Для kNN оптимизированы: `n_neighbors`, `weights`, `p`.
- Для Decision Tree оптимизированы: `max_depth`, `min_samples_split`, `min_samples_leaf`.
- Лучшие параметры найдены после 30 итераций. </b>

# 7. Обучение лучших моделей

In [ ]:
best_knn = Pipeline([
    ('preprocess', preprocessor),
    ('model', KNeighborsRegressor(**study_knn.best_params))
])
best_tree = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeRegressor(**study_tree.best_params, random_state=42))
])

best_knn.fit(X_train, y_train)
best_tree.fit(X_train, y_train)

<b>Выводы: Обучены оптимальные модели kNN и Decision Tree с подобранными параметрами на тренировочном наборе.</b>

# 8. Сравнение моделей

In [ ]:
results = pd.DataFrame([
    {'Model': 'Baseline', **baseline_test_metrics},
    {'Model': 'kNN', **regression_metrics(y_test, best_knn.predict(X_test))},
    {'Model': 'Decision Tree', **regression_metrics(y_test, best_tree.predict(X_test))}
])

In [ ]:
results

<b>Выводы: 
- kNN и Decision Tree значительно улучшили метрики по сравнению с baseline:
- RMSE ≈ 319-321
- MAE ≈ 213-215
- R2 ≈ 0.749-0.751
- kNN и Decision Tree показывают сопоставимое качество.</b>

# 9. Важность признаков для дерева

In [ ]:
preprocessor_fitted = best_tree.named_steps['preprocess']
feature_names = preprocessor_fitted.get_feature_names_out()
importances = best_tree.named_steps['model'].feature_importances_

fi = pd.Series(importances, index=feature_names).sort_values(ascending=False)

In [ ]:
fi.head(10)

<b>Выводы: - Самые важные признаки для дерева: `Temperature`, `Time_Period_Night`, `Humidity(%)`, `Time_Period_Evening`, `Functioning Day`.
- Модель учитывает как числовые, так и категориальные признаки. </b>

# 10. Кастомный трансформер для редких категорий

In [ ]:
class RareCategoryGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.frequent_categories_ = {}

    def fit(self, X, y=None):
        for col in X.columns:
            freq = X[col].value_counts(normalize=True)
            self.frequent_categories_[col] = freq[freq >= self.threshold].index.tolist()
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = X[col].apply(lambda x: x if x in self.frequent_categories_[col] else 'Other')
        return X

<b>Выводы: 
- Создан класс `RareCategoryGrouper`, объединяющий редкие категории в `Other`.
- Помогает улучшить стабильность и обобщающую способность модели.</b>

# 11. Финальный пайплайн с кастомным трансформером

In [ ]:
rare_transformer = RareCategoryGrouper(threshold=0.05)

categorical_pipeline = Pipeline([
    ('rare', rare_transformer),
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

full_preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

final_pipeline = Pipeline([
    ('preprocess', full_preprocessor),
    ('model', DecisionTreeRegressor(**study_tree.best_params, random_state=42))
])

final_pipeline.fit(X_train, y_train)
final_results = regression_metrics(y_test, final_pipeline.predict(X_test))

In [ ]:
final_results

<b>Выводы: 
- Полный пайплайн включает предобработку с `RareCategoryGrouper` и Decision Tree.
- Финальные метрики на тесте:  
- RMSE ≈ 320.64  
- MAE ≈ 212.96  
- R2 ≈ 0.749  
- Модель стабильна и готова к использованию на новых данных.

In [ ]:
# Сохраняем финальный пайплайн
joblib.dump(final_pipeline, r'C:\Users\talantr\Desktop\kNN\datasets\final_pipeline.pkl')
print('Финальный пайплайн сохранен!')

In [ ]:
# Предсказание на тестовых данных с уже обученным пайплайном
y_pred = final_pipeline.predict(X_test)

# Посчитаем метрики
final_metrics = regression_metrics(y_test, y_pred)
print('финальные метрики на тесте:', final_metrics)

Ссылка: https://github.com/TalantRahimberdiev/kNN